In [1]:
import pandas as pd

In [2]:
filepath = "YellowstoneLakesData-cleaned v1 (1).xlsx"
output_name = "YellowstoneLakesData-cleaned.csv"

In [4]:
# Read in Excel file
dictionary = pd.read_excel(filepath, sheet_name=None)

# Iterate through each DataFrame
for sheet_name, df in dictionary.items():
  # check if the first two columns are empty, then if they are drop them
  if df.iloc[:, 0].isnull().all() and df.iloc[:, 1].isnull().all():
    df = df.drop(df.columns[[0, 1]], axis=1)
    df = df.reset_index(drop=True)
  elif df.iloc[:, 0].isnull().all():
    df = df.drop(df.columns[[0]], axis=1)
    df = df.reset_index(drop=True)
  elif df.iloc[:, 1].isnull().all():
    df = df.drop(df.columns[[1]], axis=1)
    df = df.reset_index(drop=True)

  # Read down each column until the first value ignoring any that include the string "this table", then use that value as the column title.
  for column_index in range(df.shape[1]):
    for row_index in range(df.shape[0]):
      if not pd.isna(df.iloc[row_index, column_index]) and "this table" not in str(df.iloc[row_index, column_index]):
        df.columns.values[column_index] = df.iloc[row_index, column_index]
        # If the row below the first value is not blank add that value to the column title
        if not pd.isna(df.iloc[row_index+1, column_index]) and not df.iloc[row_index, column_index] == "Site":
          df.columns.values[column_index] = str(df.columns.values[column_index]) + " " + str(df.iloc[row_index+1, column_index])
        break

  # Find the empty column dividing the two datasets and use it to break the dataframe into two seperate dataframes
  for column_index in range(df.shape[1]):
    if df.iloc[:, column_index].isnull().all():
      new_df = df.iloc[:, column_index+1:].reset_index(drop=True)
      df = df.iloc[:, :column_index].reset_index(drop=True)
      break

  # Remove unnecessary columns from the dataframes
  # Iterate through the columns and find where there is only one value in the column or no values and drop those columns
  df_drop = []
  new_df_drop = []
  for column_index in range(df.shape[1]):
      if df.iloc[:, column_index].isnull().all() or df.iloc[:, column_index].notnull().sum() == 1:
          df_drop.append(column_index)
  for column_index in range(new_df.shape[1]):
      if new_df.iloc[:, column_index].isnull().all() or new_df.iloc[:, column_index].notnull().sum() == 1:
          new_df_drop.append(column_index)
  # Drop the unnecessary columns
  df = df.drop(df.columns[df_drop], axis=1)
  new_df = new_df.drop(new_df.columns[new_df_drop], axis=1)

  # Remove all unecessary rows from the dataframes using the site column
  df = df[df["Site"].notna() & (df["Site"] != "Site")].reset_index(drop=True)
  df["df_name"] = sheet_name
  new_df = new_df[new_df["Site"].notna() & (new_df["Site"] != "Site")].reset_index(drop=True)
  new_df["df_name"] = sheet_name + "2"

  # Add new_df to the bottom of df
  df = pd.concat([df, new_df], ignore_index=True)
  dictionary[sheet_name] = df

# Combine the dataframes and output a csv of the data
combine = pd.concat(dictionary.values(), ignore_index=True)
combine.to_csv(output_name, index=False)
print(f"Data saved to the Colab file explorer as {output_name}")

Data saved to the Colab file explorer as YellowstoneLakesData-cleaned.csv
